In [4]:
#TASK 1

'''
PART (A)
Variable : from (0,0) to (4,4)

Domain :
Top-left: (x−1,y−1)
Top-right: (x−1,y+1)
Bottom-left: (x+1,y−1)
Bottom-right: (x+1,y+1)

Constraints : The robot can only move diagonally,The robot must stay within the bounds of the grid,and The robot must avoid cells that contain obstacles.
'''
#PART B AND C
!pip install ortools

from ortools.sat.python import cp_model

def solve_robot_path(grid_size, start, target, obstacles, max_steps=10):
    model = cp_model.CpModel()

    x_vars = [model.NewIntVar(0, grid_size - 1, f"x_{i}") for i in range(max_steps)]
    y_vars = [model.NewIntVar(0, grid_size - 1, f"y_{i}") for i in range(max_steps)]

    model.Add(x_vars[0] == start[0])
    model.Add(y_vars[0] == start[1])
    model.Add(x_vars[max_steps - 1] == target[0])
    model.Add(y_vars[max_steps - 1] == target[1])

    for i in range(max_steps - 1):
        dx = model.NewIntVar(-1, 1, f"dx_{i}")
        dy = model.NewIntVar(-1, 1, f"dy_{i}")

        model.Add(dx == x_vars[i + 1] - x_vars[i])
        model.Add(dy == y_vars[i + 1] - y_vars[i])

        move_var = model.NewIntVar(1, 2, f"move_{i}")
        model.AddAbsEquality(move_var, dx + dy)

    for obs_x, obs_y in obstacles:
        for i in range(max_steps):
            model.AddForbiddenAssignments([x_vars[i], y_vars[i]], [(obs_x, obs_y)])

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in (cp_model.FEASIBLE, cp_model.OPTIMAL):
        return [(solver.Value(x_vars[i]), solver.Value(y_vars[i])) for i in range(max_steps)]
    else:
        return "No valid path found."

grid_size = 5
start = (0, 0)
target = (4, 4)
obstacles = [(2, 2), (3, 3)]

optimal_path = solve_robot_path(grid_size, start, target, obstacles)
print("\nOptimal Diagonal Path:", optimal_path)



Optimal Diagonal Path: [(0, 0), (1, 1), (0, 1), (1, 1), (0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (4, 4)]


In [5]:
from ortools.sat.python import cp_model

def find_largest_landmass(grid):
    rows, cols = len(grid), len(grid[0])
    model = cp_model.CpModel()

    land_vars = [[model.NewBoolVar(f"land_{r}_{c}") for c in range(cols)] for r in range(rows)]

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 0:
                model.Add(land_vars[r][c] == 0)

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 1:
                neighbors = []
                if r > 0: neighbors.append(land_vars[r-1][c])
                if r < rows - 1: neighbors.append(land_vars[r+1][c])
                if c > 0: neighbors.append(land_vars[r][c-1])
                if c < cols - 1: neighbors.append(land_vars[r][c+1])

                model.Add(sum(neighbors) >= land_vars[r][c])

    model.Maximize(sum(land_vars[r][c] for r in range(rows) for c in range(cols)))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in (cp_model.FEASIBLE, cp_model.OPTIMAL):

        landmass = [[solver.Value(land_vars[r][c]) for c in range(cols)] for r in range(rows)]
        return landmass
    else:
        return None

def compute_boundary_perimeter(landmass):
    if landmass is None:
        return 0

    rows, cols = len(landmass), len(landmass[0])
    perimeter = 0

    for r in range(rows):
        for c in range(cols):
            if landmass[r][c] == 1:
                if r == 0 or landmass[r-1][c] == 0:
                    perimeter += 1
                if r == rows - 1 or landmass[r+1][c] == 0:
                    perimeter += 1
                if c == 0 or landmass[r][c-1] == 0:
                    perimeter += 1
                if c == cols - 1 or landmass[r][c+1] == 0:
                    perimeter += 1

    return perimeter


grid = [
    [0, 1, 1, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0]
]

largest_landmass = find_largest_landmass(grid)
if largest_landmass:
    print("\nLargest Landmass:")
    for row in largest_landmass:
        print(row)
    perimeter = compute_boundary_perimeter(largest_landmass)
    print("\nBoundary Perimeter:", perimeter)
else:
    print("No landmass found.")



Largest Landmass:
[0, 1, 1, 0, 0]
[0, 1, 1, 1, 0]
[0, 0, 1, 1, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 0]

Boundary Perimeter: 14


In [ ]:
#Task3
from ortools.sat.python import cp_model

def tsp_csp(distance_matrix):
    num_cities = len(distance_matrix)

    model = cp_model.CpModel()

    x = {}
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                x[i, j] = model.NewBoolVar(f'x_{i}_{j}')

    u = {}
    for i in range(1, num_cities):
        u[i] = model.NewIntVar(1, num_cities - 1, f'u_{i}')

    for i in range(num_cities):
        model.Add(sum(x[i, j] for j in range(num_cities) if i != j) == 1)
        model.Add(sum(x[j, i] for j in range(num_cities) if i != j) == 1)


    for i in range(1, num_cities):
        for j in range(1, num_cities):
            if i != j:
                model.Add(u[i] + 1 <= u[j] + num_cities * (1 - x[i, j]))

    total_cost = sum(distance_matrix[i][j] * x[i, j] for i in range(num_cities) for j in range(num_cities) if i != j)
    model.Minimize(total_cost)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        path = []
        current = 0
        visited = set()

        while len(visited) < num_cities:
            visited.add(current)
            for j in range(num_cities):
                if j != current and solver.Value(x[current, j]) == 1:
                    path.append((current, j))
                    current = j
                    break

        print("Optimal TSP Path:", path)
        print("Minimum Travel Cost:", solver.ObjectiveValue())
    else:
        print("No feasible solution found!")

import random
random.seed(42)
num_cities = 10
distance_matrix = [[random.randint(10, 100) if i != j else 0 for j in range(num_cities)] for i in range(num_cities)]

tsp_csp(distance_matrix)


Optimal TSP Path: [(0, 8), (8, 4), (4, 1), (1, 2), (2, 3), (3, 5), (5, 9), (9, 6), (6, 7), (7, 0)]
Minimum Travel Cost: 253.0


In [20]:
from ortools.sat.python import cp_model

def warehouse_robot_csp():
    model = cp_model.CpModel()

    grid_size = 6
    num_robots = 5
    num_packages = 10
    max_steps = 15

    x = {}
    y = {}

    for r in range(num_robots):
        for t in range(max_steps):
            x[r, t] = model.NewIntVar(0, grid_size - 1, f"x_{r}_{t}")
            y[r, t] = model.NewIntVar(0, grid_size - 1, f"y_{r}_{t}")

    # Package status (0: waiting, 1: in transit, 2: delivered)
    package_status = {}
    for p in range(num_packages):
        package_status[p] = model.NewIntVar(0, 2, f"package_{p}")

    for r in range(num_robots):
        for t in range(1, max_steps):
            dx = model.NewIntVar(-1, 1, f"dx_{r}_{t}")
            dy = model.NewIntVar(-1, 1, f"dy_{r}_{t}")

            model.Add(dx == x[r, t] - x[r, t - 1])
            model.Add(dy == y[r, t] - y[r, t - 1])

            model.AddAbsEquality(model.NewIntVar(0, 1, f"abs_dx_{r}_{t}"), dx)
            model.AddAbsEquality(model.NewIntVar(0, 1, f"abs_dy_{r}_{t}"), dy)

    for t in range(max_steps):
        for r1 in range(num_robots):
            for r2 in range(r1 + 1, num_robots):
                not_same_x = model.NewBoolVar(f"not_same_x_{r1}_{r2}_{t}")
                not_same_y = model.NewBoolVar(f"not_same_y_{r1}_{r2}_{t}")

                model.Add(x[r1, t] != x[r2, t]).OnlyEnforceIf(not_same_x)
                model.Add(y[r1, t] != y[r2, t]).OnlyEnforceIf(not_same_y)

                model.AddBoolOr([not_same_x, not_same_y])

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("\nOptimal Delivery Plan:")
        for t in range(max_steps):
            for r in range(num_robots):
                print(f"Robot {r} at step {t}: ({solver.Value(x[r, t])}, {solver.Value(y[r, t])})")
    else:
        print("No feasible solution found.")


warehouse_robot_csp()



Optimal Delivery Plan:
Robot 0 at step 0: (1, 0)
Robot 1 at step 0: (2, 3)
Robot 2 at step 0: (0, 1)
Robot 3 at step 0: (0, 0)
Robot 4 at step 0: (2, 2)
Robot 0 at step 1: (0, 1)
Robot 1 at step 1: (1, 2)
Robot 2 at step 1: (1, 0)
Robot 3 at step 1: (1, 1)
Robot 4 at step 1: (2, 3)
Robot 0 at step 2: (0, 2)
Robot 1 at step 2: (0, 1)
Robot 2 at step 2: (1, 1)
Robot 3 at step 2: (2, 0)
Robot 4 at step 2: (1, 3)
Robot 0 at step 3: (0, 1)
Robot 1 at step 3: (1, 1)
Robot 2 at step 3: (0, 2)
Robot 3 at step 3: (1, 0)
Robot 4 at step 3: (2, 3)
Robot 0 at step 4: (1, 0)
Robot 1 at step 4: (0, 1)
Robot 2 at step 4: (0, 2)
Robot 3 at step 4: (0, 0)
Robot 4 at step 4: (2, 2)
Robot 0 at step 5: (2, 0)
Robot 1 at step 5: (0, 0)
Robot 2 at step 5: (0, 2)
Robot 3 at step 5: (1, 1)
Robot 4 at step 5: (2, 1)
Robot 0 at step 6: (2, 1)
Robot 1 at step 6: (0, 1)
Robot 2 at step 6: (1, 1)
Robot 3 at step 6: (2, 2)
Robot 4 at step 6: (1, 0)
Robot 0 at step 7: (1, 0)
Robot 1 at step 7: (0, 0)
Robot 2 at ste

In [19]:
from ortools.sat.python import cp_model

def solve_sudoku(grid):
    model = cp_model.CpModel()

    SIZE = 9
    SUBGRID = 3

    cells = [[model.NewIntVar(1, 9, f'cell_{r}_{c}') for c in range(SIZE)] for r in range(SIZE)]

    for r in range(SIZE):
        for c in range(SIZE):
            if grid[r][c] != 0:
                model.Add(cells[r][c] == grid[r][c])

    for r in range(SIZE):
        model.AddAllDifferent(cells[r])

    for c in range(SIZE):
        model.AddAllDifferent([cells[r][c] for r in range(SIZE)])

    for r in range(0, SIZE, SUBGRID):
        for c in range(0, SIZE, SUBGRID):
            subgrid_cells = [cells[r + dr][c + dc] for dr in range(SUBGRID) for dc in range(SUBGRID)]
            model.AddAllDifferent(subgrid_cells)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        solved_grid = [[solver.Value(cells[r][c]) for c in range(SIZE)] for r in range(SIZE)]
        for row in solved_grid:
            print(row)
    else:
        print("No solution found.")

sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

solve_sudoku(sudoku_grid)


[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
